In [1]:
import struct

# Helper function to read the control octet and determine the type and tag
def read_control_octet(data):
    if not data:
        return None, None, data
    control_octet = data[0]
    tag_control = (control_octet >> 5) & 0x07  # Upper 3 bits
    element_type = control_octet & 0x1F        # Lower 5 bits
    return tag_control, element_type, data[1:]

# Function to read the tag based on tag control
def read_tag(data, tag_control):
    tag_sizes = {0: 0, 1: 1, 2: 2, 3: 4, 4: 2, 5: 4, 6: 6, 7: 8}
    tag_size = tag_sizes.get(tag_control, 0)
    tag = data[:tag_size]
    return int.from_bytes(tag, 'little'), data[tag_size:]

# Function to read the length and value based on element type
def read_length_and_value(data, element_type):
    # Mapping element types to their value sizes
    type_size = {
        0: 1, 1: 2, 2: 4, 3: 8, 4: 1, 5: 2, 6: 4, 7: 8,
        10: 4, 11: 8, 12: 1, 13: 2, 14: 4, 15: 8,
        16: 1, 17: 2, 18: 4, 19: 8
    }
    length = type_size.get(element_type, None)
    
    if length is None:
        # Container or special types
        if element_type in {21, 22, 23}:  # Structures, Arrays, Lists
            return None, data
        elif element_type == 24:  # End-of-container
            return None, data
        else:
            raise ValueError("Unsupported element type")
    else:
        value = data[:length]
        return value, data[length:]

# Function to read and parse TLV data recursively
def parse_tlv(data):
    elements = []
    while data:
        tag_control, element_type, data = read_control_octet(data)
        if element_type == 24:  # End-of-container
            break
        tag, data = read_tag(data, tag_control)
        value, data = read_length_and_value(data, element_type)
        if element_type in {21, 22, 23}:  # Recursive parsing for containers
            value, data = parse_tlv(data)
        elements.append({'tag': tag, 'type': element_type, 'value': value})
    return elements, data

# Example usage
data = b'\x15\x18'  # Example data for an empty structure
parsed_data, remaining = parse_tlv(data)
print(parsed_data)

# The parse_tlv function now handles nested structures by recursively calling itself when a container type is encountered.


[{'tag': 0, 'type': 21, 'value': []}]
